In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import os

In [2]:
ts_indicator = "C:\\quanttime\\data\\finance\\ts\\indicator\\" 
ts_stock_name = "C:\\quanttime\\data\\basic_info\\all_stock_info_ts.csv"

In [3]:
# ts_code与stock name的关系
df_stock_name = pd.read_csv(ts_stock_name,index_col=["ts_code"],usecols=["ts_code","name"],encoding="gbk")

In [4]:
# 指标的英文缩写与中文的对应关系
indicator_name = pd.read_csv(ts_indicator+'field_name.txt',encoding='gbk')
indicator_name['field_name_en'] = indicator_name['ts_code:ts代码'].apply(lambda x: str(x).split(':')[0])
indicator_name['field_name_en_cn'] = indicator_name['ts_code:ts代码'].apply(lambda x: str(x).split(':')[1])
indicator_name = indicator_name.set_index("field_name_en")
indicator_name = indicator_name[["field_name_en_cn"]]


In [5]:
all_indicator_file = os.listdir(ts_indicator)
# 只挑选出股票csv指标文件
all_csv_file = [d for d in all_indicator_file if len(d)==13 and 'csv' in d ]

In [6]:
'''
将所有stock的指标集中到一个df
并计算5年，3年的平均roe
'''
df_indicator = pd.DataFrame()
list_roe_result = []
year_5_mean = [20181231,20171231,20161231,20151231,20141231]
year_3_mean = [20181231,20171231,20161231]
# 计算了5年，3年roe平均以及std
for stock_indicator in all_csv_file:
    file_path = ts_indicator + stock_indicator
    df_tmp_indicator = pd.read_csv(file_path)
    #print(df_tmp_indicator)
    df_indicator = df_indicator.append(df_tmp_indicator,ignore_index=True)
    list_roe_5 = []
    list_roe_3 = []
    df_tmp_indicator = pd.read_csv(file_path, index_col=["end_date"])
    for year in year_5_mean:
        if year in df_tmp_indicator.index:
            list_roe_5.append(df_tmp_indicator.loc[year,["roe"]]["roe"])
    for year in year_3_mean:
        if year in df_tmp_indicator.index:
            list_roe_3.append(df_tmp_indicator.loc[year,["roe"]]["roe"])
    #print([stock_indicator[0:9], list_roe_5, list_roe_3])
    tmp_5_mean = round(sum(list_roe_5)/len(list_roe_5),2)
    tmp_3_mean = round(sum(list_roe_3)/len(list_roe_3),2)
    tmp_5_std = round(np.std(list_roe_5),2)
    tmp_3_std = round(np.std(list_roe_3),2)
    list_roe_result.append([stock_indicator[0:9], tmp_5_mean, tmp_3_mean, tmp_5_std, tmp_3_std])
roe_columns_name = ["ts_code", "roe_5", "roe_3", "roe_5_std", "roe_3_std"]
df_roe_mean = pd.DataFrame(data=list_roe_result, columns=roe_columns_name)
        
df_roe_mean

,ts_code,roe_5,roe_3,roe_5_std,roe_3_std
0,000001.SZ,13.07,11.37,2.21,0.75
1,000002.SZ,20.84,21.96,1.87,1.63
2,000004.SZ,7.02,9.57,18.02,22.87
3,000005.SZ,4.26,6.90,6.07,4.10
4,000006.SZ,13.76,15.56,2.42,0.73
5,000007.SZ,-9.45,-14.17,30.73,38.63
6,000008.SZ,8.27,9.77,4.54,3.78
7,000009.SZ,8.21,4.07,6.04,0.99
8,000010.SZ,-32.89,-51.38,38.40,39.56
9,000011.SZ,17.42,19.35,5.58,3.13


In [9]:
df_roe_mean = df_roe_mean.set_index(["ts_code"])

In [7]:
# 指标筛选，将所有需要筛选的指标放到该list
filter_columns_name = ["roe", "roe_dt","roe_waa", "roe_yearly", "debt_to_assets", "ocfps", "ts_code","end_date"]
df_analyse = df_indicator[filter_columns_name]
#df_analyse

In [8]:
df_analyse = df_analyse.set_index(["ts_code"])
#

In [10]:
df_analyse = df_analyse[df_analyse["end_date"]==20181231]
#df_analyse = df_analyse.sort_values(by=["roe"],ascending=False)
#df_analyse = df_analyse[df_analyse["roe"]>=15]
df_analyse = pd.merge(df_analyse,df_stock_name,left_index=True,right_index=True)
df_analyse = pd.merge(df_analyse,df_roe_mean,left_index=True,right_index=True)


In [11]:
pd.set_option('display.max_rows', None)

In [18]:
# 再增加一个审计师为四大的标准
audit_file_path = "C:\\quanttime\\data\\finance\\audit\\"
all_audit_file = os.listdir(audit_file_path)
df_audit = pd.DataFrame()
columns_name = ["ts_code", "end_date", "audit_result", "audit_agency"]
audit_top = [u'普华永道中天会计师事务所', u'安永华明会计师事务所', u'毕马威华振会计师事务所', u'德勤华永会计师事务所']
for audit in all_audit_file:
    df_data = pd.read_csv(audit_file_path+audit, encoding="gbk",usecols=columns_name)
    df_audit = df_audit.append(df_data.iloc[0])


In [21]:
# 筛选出四大
df_audit_phyd = df_audit[df_audit["audit_agency"]==u'普华永道中天会计师事务所']
df_audit_ay = df_audit[df_audit["audit_agency"]==u'安永华明会计师事务所']
df_audit_pwc = df_audit[df_audit["audit_agency"]==u'毕马威华振会计师事务所']
df_audit_dq = df_audit[df_audit["audit_agency"]==u'德勤华永会计师事务所']
df_audit_top = pd.concat([df_audit_phyd,df_audit_ay,df_audit_pwc,df_audit_dq])
df_audit_top

,audit_agency,audit_result,end_date,ts_code
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000001.SZ
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000039.SZ
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000099.SZ
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000333.SZ
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000418.SZ
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000539.SZ
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000550.SZ
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000608.SZ
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000652.SZ
0,普华永道中天会计师事务所,标准无保留意见,20181231.0,000708.SZ


In [22]:
df_analyse = df_analyse[df_analyse["debt_to_assets"]<=40.5]

In [23]:
df_analyse = df_analyse[df_analyse["ocfps"]>0]
df_analyse = df_analyse[df_analyse["roe_5_std"]<=4]

In [24]:
df_analyse = df_analyse[df_analyse["roe_5"]>=15]

In [25]:
df_analyse = pd.merge(df_analyse,df_audit_top,left_index=True, right_on="ts_code")

In [26]:
df_analyse.sort_values(by=["roe_5"],ascending=False)

,roe,roe_dt,roe_waa,roe_yearly,debt_to_assets,ocfps,end_date_x,name,roe_5,roe_3,roe_5_std,roe_3_std,audit_agency,audit_result,end_date_y,ts_code
0,33.4162,32.3282,33.99,33.4162,40.1992,0.9877,20181231,海康威视,34.45,33.98,0.80,0.43,德勤华永会计师事务所,标准无保留意见,20181231.0,002415.SZ
0,34.0622,32.1837,32.66,34.0622,31.0565,2.2205,20181231,海天味业,32.88,32.27,2.30,1.54,毕马威华振会计师事务所,标准无保留意见,20181231.0,603288.SH
0,35.6237,33.6953,34.06,35.6237,37.3578,1.5744,20181231,双汇发展,29.49,31.34,3.31,3.09,安永华明会计师事务所,标准无保留意见,20181231.0,000895.SZ
0,26.3031,25.3973,26.63,26.3031,5.9053,0.7098,20181231,我武生物,24.04,24.24,2.36,2.41,安永华明会计师事务所,标准无保留意见,20181231.0,300357.SZ
0,19.0555,16.4139,16.42,19.0555,33.0484,1.0430,20181231,爱婴室,23.96,24.39,3.16,3.81,安永华明会计师事务所,标准无保留意见,20181231.0,603214.SH
0,19.7184,18.1129,19.72,19.7184,18.3198,1.5428,20181231,东阿阿胶,23.28,22.09,2.02,1.81,安永华明会计师事务所,标准无保留意见,20181231.0,000423.SZ
0,16.1130,14.5903,14.57,16.1130,5.4608,2.0582,20181231,华宝股份,21.72,21.14,2.99,3.62,普华永道中天会计师事务所,标准无保留意见,20181231.0,300741.SZ
0,22.1830,21.7692,22.06,22.1830,34.9961,0.7459,20181231,长飞光纤,21.61,22.58,3.30,3.43,毕马威华振会计师事务所,标准无保留意见,20181231.0,601869.SH
0,18.8139,12.9717,23.98,18.8139,19.8612,1.4084,20181231,药明康德,18.13,20.41,3.52,1.23,德勤华永会计师事务所,标准无保留意见,20181231.0,603259.SH
0,20.4487,20.7764,20.56,20.4487,30.7781,1.3942,20181231,中国国旅,16.83,18.22,2.44,2.21,毕马威华振会计师事务所,标准无保留意见,20181231.0,601888.SH


In [27]:
df_analyse.sort_values(by=["roe_5"],ascending=False).to_csv(r"c:\quanttime\data\tmp\indicator_roe.csv",encoding="gbk")

In [50]:
df_analyse.loc["600519.SH"]

roe                34.4643
roe_dt             34.8382
roe_waa              34.46
roe_yearly         34.4643
debt_to_assets     26.5493
ocfps              32.9448
end_date          20181231
name                  贵州茅台
roe_5                30.05
roe_3                30.62
roe_5_std              3.9
roe_3_std             4.41
Name: 600519.SH, dtype: object